# word2vec: How To Prep Word Vectors For Modeling

### Train Our Own Model

In [2]:
# Read in the data, clean it, split it into train and test sets, and then train a word2vec model
import gensim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', 100)

messages = pd.read_csv('../../../data/spam.csv', encoding='latin-1')
messages = messages.drop(labels = ["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis = 1)
messages.columns = ["label", "text"]


messages['text_clean'] = messages['text'].apply(lambda x: gensim.utils.simple_preprocess(x))
X_train, X_test, y_train, y_test = train_test_split(messages['text_clean'],
                                                    messages['label'], test_size=0.2)

w2v_model = gensim.models.Word2Vec(X_train,
                                   vector_size=100,
                                   window=5,
                                   min_count=2)

### Prep Word Vectors

In [5]:
# Generate a list of words the word2vec model learned word vectors for
w2v_model.wv.index_to_key

['to',
 'you',
 'the',
 'and',
 'in',
 'is',
 'me',
 'my',
 'it',
 'for',
 'your',
 'of',
 'call',
 'have',
 'that',
 'on',
 'now',
 'are',
 'can',
 'so',
 'not',
 'do',
 'but',
 'we',
 'or',
 'get',
 'at',
 'ur',
 'just',
 'no',
 'be',
 'will',
 'if',
 'with',
 'this',
 'lt',
 'gt',
 'up',
 'how',
 'when',
 'go',
 'what',
 'free',
 'from',
 'ok',
 'all',
 'out',
 'll',
 'good',
 'know',
 'got',
 'day',
 'then',
 'like',
 'am',
 'was',
 'there',
 'he',
 'only',
 'its',
 'come',
 'time',
 'love',
 'send',
 'text',
 'want',
 'txt',
 'about',
 'as',
 'home',
 'need',
 'one',
 'today',
 'going',
 'by',
 'see',
 'lor',
 'she',
 'stop',
 'our',
 'sorry',
 'da',
 'don',
 'back',
 'still',
 'hi',
 'take',
 'dont',
 'any',
 'reply',
 'phone',
 'tell',
 'new',
 'did',
 'pls',
 'week',
 'they',
 'think',
 'please',
 'mobile',
 'here',
 'been',
 'oh',
 'her',
 'later',
 'where',
 'some',
 'much',
 'ì_',
 'great',
 'msg',
 'dear',
 'hey',
 'happy',
 'hope',
 'more',
 'an',
 'claim',
 'who',
 'well'

In [7]:
# Generate aggregated sentence vectors based on the word vectors for each word in the sentence
w2v_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in w2v_model.wv.index_to_key])
                     for ls in X_test])

c:\Users\shadm\Documents\Codes\advanced-nlp-with-python-for-machine-learning\venv\lib\site-packages\ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
# Why is the length of the sentence different than the length of the sentence vector?
for i, v in enumerate(w2v_vect):
    print(len(X_test.iloc[i]), len(v))

14 13
22 18
36 35
7 7
10 10
27 25
38 31
30 29
6 5
4 3
6 6
21 21
11 10
6 6
22 22
31 30
6 6
8 8
1 1
9 9
5 5
9 6
3 3
13 11
5 5
3 3
15 15
5 5
27 25
22 20
7 7
5 4
31 29
23 22
18 18
10 10
6 6
25 22
25 23
12 12
13 12
9 9
11 11
7 7
14 14
6 3
6 6
16 16
30 29
6 6
12 11
27 26
5 5
10 9
9 9
26 25
9 9
10 9
13 10
37 35
5 4
19 19
18 16
6 6
16 15
11 11
9 5
5 5
4 2
14 11
32 30
8 8
10 10
44 40
6 6
7 7
21 21
10 10
27 27
8 8
3 3
18 18
6 6
6 6
32 26
23 22
4 4
7 5
11 11
15 15
23 20
11 8
30 28
28 27
3 1
20 19
9 8
6 6
22 18
50 44
28 27
6 5
6 6
10 10
7 5
21 21
25 25
5 5
2 2
8 8
5 5
13 10
26 26
6 6
9 8
9 9
23 18
8 8
6 6
7 7
23 21
7 6
4 4
4 3
6 6
14 14
8 7
4 2
25 25
18 11
1 1
37 29
17 17
24 20
19 18
8 7
24 19
21 17
5 5
5 4
16 15
4 3
4 4
9 9
11 10
13 12
17 17
9 9
9 8
18 18
29 28
13 13
5 5
26 25
16 15
26 22
16 15
25 14
19 19
4 4
14 13
16 12
7 7
23 21
16 15
5 5
9 9
2 2
24 21
26 25
13 13
7 5
29 23
40 38
21 20
23 22
9 7
22 21
20 20
7 7
8 7
5 5
12 4
4 4
7 7
10 8
6 4
10 7
6 5
12 11
25 25
6 6
12 12
6 4
7 7
4 3
3 3
6 6
25

In [9]:
# Compute sentence vectors by averaging the word vectors for the words contained in the sentence
w2v_vect_avg = []

for vect in w2v_vect:
    if len(vect)!=0:
        w2v_vect_avg.append(vect.mean(axis=0))
    else:
        w2v_vect_avg.append(np.zeros(100))

In [10]:
# Are our sentence vector lengths consistent?
for i, v in enumerate(w2v_vect_avg):
    print(len(X_test.iloc[i]), len(v))

14 100
22 100
36 100
7 100
10 100
27 100
38 100
30 100
6 100
4 100
6 100
21 100
11 100
6 100
22 100
31 100
6 100
8 100
1 100
9 100
5 100
9 100
3 100
13 100
5 100
3 100
15 100
5 100
27 100
22 100
7 100
5 100
31 100
23 100
18 100
10 100
6 100
25 100
25 100
12 100
13 100
9 100
11 100
7 100
14 100
6 100
6 100
16 100
30 100
6 100
12 100
27 100
5 100
10 100
9 100
26 100
9 100
10 100
13 100
37 100
5 100
19 100
18 100
6 100
16 100
11 100
9 100
5 100
4 100
14 100
32 100
8 100
10 100
44 100
6 100
7 100
21 100
10 100
27 100
8 100
3 100
18 100
6 100
6 100
32 100
23 100
4 100
7 100
11 100
15 100
23 100
11 100
30 100
28 100
3 100
20 100
9 100
6 100
22 100
50 100
28 100
6 100
6 100
10 100
7 100
21 100
25 100
5 100
2 100
8 100
5 100
13 100
26 100
6 100
9 100
9 100
23 100
8 100
6 100
7 100
23 100
7 100
4 100
4 100
6 100
14 100
8 100
4 100
25 100
18 100
1 100
37 100
17 100
24 100
19 100
8 100
24 100
21 100
5 100
5 100
16 100
4 100
4 100
9 100
11 100
13 100
17 100
9 100
9 100
18 100
29 100
13 100
5 100
2